In [1]:
from data_common.notebook import *

# Dataset analysis

## Counts by authority

In [2]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df["modern"] = (
    df["end-date"]
    .isna()
    .map({True: "Current authorities", False: "Former authorities"})
)

(
    df.pivot_table(
        "local-authority-code",
        index="local-authority-type-name",
        columns="modern",
        aggfunc="count",
    )
    .reset_index()
    .rename(
        columns={
            "local-authority-code": "Count",
            "local-authority-type-name": "Authority type",
        }
    )
    .fillna(0)
    .style.format({"Current authorities": int, "Former authorities": int})
    .hide(axis="index")
)

## Unitary/lower tier and total counts

In [3]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df = df[df["current-authority"]]

df["lower-or-unitary"] = df["lower-or-unitary"].map({True: "Yes", False: "No"})

df = (
    df.pivot_table("local-authority-code", index="lower-or-unitary", aggfunc="count")
    .reset_index()
    .rename(
        columns={
            "local-authority-code": "Count",
            "lower-or-unitary": "Lower or unitary?",
        }
    )
    .fillna(0)
)

summary_row = pd.DataFrame(
    {"Lower or unitary?": "All", "Count": df["Count"].sum()}, index=[50]
)

df = pd.concat([df, summary_row])
df.style.hide(axis="index")

## Incomplete lookups

This are optional columns, and not entirely populated.

In [4]:
df = pd.read_csv(Path("data", "uk_local_authorities.csv"))
df = df[df["current-authority"]]
items = []

cols = list(df.columns)
cols = cols[cols.index("BS-6879") :]


for c in cols:
    count = (~df[c].isna()).sum()
    items.append({"column": c, "complete": count})

pt = pd.DataFrame(items).sort_values("complete", ascending=False)
pt["%"] = pt["complete"] / len(df)
pt = pt[pt["%"] < 1]
pt.style.format({"%": "{:.1%}".format}).hide(axis="index")